In [ ]:
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

prefix = '/mnt/data1/mimic/iii'
def dpath(fname):
    return os.path.join(prefix, fname)

def cleaned(fname):
    return os.path.join(prefix, 'cleaned', fname)

In [ ]:
uidcols = ['SUBJECT_ID', 'HADM_ID', 'CHARTTIME']

# Extract Progress Notes

In [ ]:
usecols = ['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'CATEGORY', 'DESCRIPTION', 'ISERROR', 'TEXT']
dtypes =  [int,          float,     object,      object,      str,        str,           float,     str]

notes = pd.read_csv(
    dpath('NOTEEVENTS.csv.gz'),
    usecols=usecols,
    dtype=dict(zip(usecols, dtypes)),
    parse_dates=['CHARTDATE', 'CHARTTIME'],
)
notes = notes.dropna(subset='HADM_ID')
notes['HADM_ID'] = notes['HADM_ID'].astype(int)

In [ ]:
progress_notes = notes[
    notes['CATEGORY'].isin({'Physician '}) &
    notes['DESCRIPTION'].str.contains('Physician|Attending|Fellow|Resident|PGY', case=False, regex=True) &
    notes['DESCRIPTION'].str.contains('prog', case=False) &
    notes['ISERROR'].isna()
]
progress_notes = progress_notes.sort_values(uidcols)

In [ ]:
print('Number of progress notes:', len(progress_notes))
print('Number of unique progress note chart times:', len(progress_notes.drop_duplicates(uidcols)))
print('Unique encounters with progress notes:', len(progress_notes['HADM_ID'].drop_duplicates()))
assert not progress_notes['CHARTTIME'].isna().any()

In [ ]:
progress_notes[uidcols + ['TEXT']].to_csv(cleaned('Progress_Notes.csv'), index=False)

# Load CHARTEVENTS

In [ ]:
_items = pd.read_csv(dpath('D_ITEMS.csv.gz'))
items = _items.dropna(axis=0, subset='LABEL')

parquet_file = dpath('CHARTEVENTS.parquet')
if not os.path.exists(parquet_file):
    chunksize = 1_000_000
    usecols = ['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUE', 'VALUENUM', 'VALUEUOM']
    dtypes =  [int,          int,       int,      object,      str,     float,      str]

    chunks = []
    it = pd.read_csv(
        dpath('CHARTEVENTS.csv.gz'),
        chunksize=chunksize,
        usecols=usecols,
        dtype=dict(zip(usecols, dtypes)),
        parse_dates=['CHARTTIME'],
    )

    for chunk in tqdm(it):
        chunks.append(chunk)

    chart = pd.concat(chunks)

    _df = chart.merge(items[['ITEMID', 'LABEL', 'DBSOURCE']], how='left', on='ITEMID')
    _df = _df.sort_values(uidcols)
    _df.to_parquet(parquet_file)

df = pd.read_parquet(parquet_file)

In [ ]:
meas_count = df['ITEMID'].value_counts().reset_index().merge(items[['ITEMID', 'LABEL', 'DBSOURCE']], how='left', on='ITEMID')[['ITEMID', 'LABEL', 'DBSOURCE', 'count']]
def lookup(term, case=False, min_count=0):
    mask = meas_count['LABEL'].str.contains(term, case=case, regex=True)
    mask2 = meas_count['count'] >= min_count
    ret = meas_count.loc[mask & mask2]
    return ret

def values_of(val):
    return df[df['ITEMID'] == val]['VALUENUM'].value_counts()

def units_of(val):
    return df[df['ITEMID'] == val]['VALUEUOM'].value_counts()

def units(_df):
    return _df['VALUEUOM'].value_counts()

from collections.abc import Iterable
def filter(vals):
    if isinstance(vals, Iterable):
        vals = set(vals)
    else:
        vals = {vals}
    return df[df['ITEMID'].isin(vals)]

def clean(_df, name, asint=False, xlim=[0, 200]):
    if 'VALUEUOM' in _df.columns:
        print('=== Units ===')
        print(units(_df))
        print()
    if 'LABEL' in _df.columns:
        ax = sns.boxplot(_df, x='VALUENUM', y=_df['LABEL'] + ' (' + _df['ITEMID'].astype(str) + ')')
        ax.set_ylabel('LABEL (ITEMID)')
    else:
        ax = sns.boxplot(_df, x='VALUENUM')
    ax.set_xlim(xlim)
    ret = _df[uidcols + ['VALUENUM']].dropna(axis=0).rename(columns={'VALUENUM': name})
    ret = ret.groupby(uidcols)[name].mean()
    if asint:
        ret = ret.round().astype(int)
    ret = ret.reset_index()
    return ret

# Extract Vitals

### Glascow Coma Scale

In [ ]:
lookup('GCS')

In [ ]:
gcs_cv_itemid = 198
gcs_mv_itemids = {220739, 223900, 223901}

In [ ]:
gcs_cv_df = filter(gcs_cv_itemid)
gcs_mv_df = filter(gcs_mv_itemids)

In [ ]:
gcs_cv = gcs_cv_df.set_index(uidcols)['VALUENUM']
gcs_mv = pd.pivot(gcs_mv_df, index=uidcols, columns='LABEL', values='VALUENUM').sum(axis=1, skipna=False)

In [ ]:
gcs_df = pd.concat([gcs_cv, gcs_mv]).sort_index()
gcs_df.name = 'VALUENUM'
gcs_df = gcs_df.reset_index()

In [ ]:
gcs = clean(gcs_df, 'GCS', asint=True, xlim=[0, 20])

In [ ]:
gcs.to_csv(cleaned('GCS.csv'), index=False)

### Heart Rate

In [ ]:
lookup('heart rate|HR', min_count=10_000)

In [ ]:
hr_itemids = {211, 220045}
hr_df = filter(hr_itemids)

In [ ]:
hr = clean(hr_df, 'HR', asint=True, xlim=[-50, 250])

In [ ]:
hr.to_csv(cleaned('HR.csv'), index=False)

### Respiratory Rate

In [ ]:
rr_items = lookup('resp.*rate', min_count=10_000)
rr_items

In [ ]:
rr_itemids = set(rr_items['ITEMID'])
rr_df = filter(rr_itemids)

In [ ]:
rr = clean(rr_df, 'RR', asint=True, xlim=[-10, 200])

In [ ]:
rr.to_csv(cleaned('RR.csv'), index=False)

### Blood Pressure

In [ ]:
bp_item_df = lookup('blood pressure|bp', min_count=10_000)
bp_item_df = bp_item_df[
    (~bp_item_df['LABEL'].str.contains('alarm', case=False)) &
    (~bp_item_df['LABEL'].str.contains('IABP', case=False)) &
    (~bp_item_df['LABEL'].str.contains('rate', case=False)) &
    (~bp_item_df['LABEL'].str.contains('BP UAC', case=True)) &
    (~bp_item_df['LABEL'].str.contains('BP PAL', case=True)) &
    (~bp_item_df['LABEL'].str.contains('BP Cuff', case=True)) # weird range and weird units (cc/min)?
]
bp_item_df

In [ ]:
sbp_itemids = [51,   220050]
dbp_itemids = [8368, 220051]
# mbp_itemids = [52,   220052]

snbp_itemids = [455,  220179]
dnbp_itemids = [8441, 220180]
# mnbp_itemids = [456,  220181]

In [ ]:
sbp_df = filter(sbp_itemids)
dbp_df = filter(dbp_itemids)

snbp_df = filter(snbp_itemids)
dnbp_df = filter(dnbp_itemids)

In [ ]:
sbp = clean(sbp_df, 'VALUENUM', asint=True)

In [ ]:
dbp = clean(dbp_df, 'VALUENUM', asint=True)

In [ ]:
snbp = clean(snbp_df, 'VALUENUM', asint=True)

In [ ]:
dnbp = clean(dnbp_df, 'VALUENUM', asint=True)

In [ ]:
bp_df = pd.concat([sbp, dbp])
bp_df['LABEL'] = ['SBP'] * len(sbp) + ['DBP'] * len(dbp)

nbp_df = pd.concat([snbp, dnbp])
nbp_df['LABEL'] = ['SBP'] * len(snbp) + ['DBP'] * len(dnbp)

In [ ]:
bp_pivot = pd.pivot(bp_df, index=uidcols, columns='LABEL', values='VALUENUM')
nbp_pivot = pd.pivot(nbp_df, index=uidcols, columns='LABEL', values='VALUENUM')

In [ ]:
bp = bp_pivot.dropna(axis=0, subset=['DBP', 'SBP']).astype(int).reset_index()
bp.columns.name = None

nbp = nbp_pivot.dropna(axis=0, subset=['DBP', 'SBP']).astype(int).reset_index()
nbp.columns.name = None

In [ ]:
bp.to_csv(cleaned('BP.csv'), index=False)
nbp.to_csv(cleaned('NBP.csv'), index=False)

### SpO2

In [ ]:
lookup('pulse.*ox|spo2')

In [ ]:
spo2_itemids = {220277, 646}
spo2_df = filter(spo2_itemids)

In [ ]:
spo2 = clean(spo2_df, 'SpO2', asint=True)

In [ ]:
spo2.to_csv(cleaned('SpO2.csv'), index=False)

# Extract Labs

### HCO3

In [ ]:
co2_items = lookup('HCO3|CO2|Bicarb|Carbon', min_count=1000)
co2_items

In [ ]:
filter(co2_items['ITEMID'])

In [ ]:
filter(777)

In [ ]:
co2_itemids = {227443, 787, 3810, 3808, 225698}
co2_df = filter(co2_itemids)

In [ ]:
ax = sns.boxplot(co2_df, x='VALUENUM', y='LABEL')
ax.set_xlim([0, 100])

In [ ]:
co2_df['VALUEUOM'].value_counts()

In [ ]:
co2 = clean(co2_df, 'HCO3')
co2

In [ ]:
co2.to_csv(dpath('HCO3.csv'), index=False)

### Cl

In [ ]:
lookup('chlor')

In [ ]:
lookup('Cl')

In [ ]:
cl_itemids = {788, 220602, 1523, 3747, 226536}
cl_df = filter(cl_itemids)

In [ ]:
ax = sns.boxplot(cl_df, x='VALUENUM', y='LABEL')
ax.set_xlim([0, 200])

In [ ]:
units(cl_df)

In [ ]:
cl = clean(cl_df, 'Cl')
cl

In [ ]:
cl.to_csv(dpath('Cl.csv'), index=False)

### K

### Na

### Anion Gap

### Ca

### Glucose

### Creatinine

### eGFR

### BUN

### Total Protein

### Albumin

### Total Bilirubin

### Conjugated Bilirubin

### Unconjugated Bilirubin

### Alk Phos

### AST

### ALT